# Librairies

In [53]:
import project_functions as pf
import pandas as pd

# Portfolio Analysis


Importons les données :

In [54]:
# récupérons les données des prix sauvegardés dans le fichier prices.csv
prices = pd.read_csv('datas/prices.csv', sep=',')
prices = prices.set_index('Date')

## Analyse des returns

Nous ne prenons uniquement les assets avec des returns anualisés positifs.

In [55]:
# Calculons les rendements journaliers
daily_returns = pf.daily_asset_return(prices)

# Calculons les rendements annuels
annualy_returns = pf.annualy_asset_return(daily_returns)

# On ne concerne uniquement les index avec rendements annuels positifs
positive_returns = annualy_returns[annualy_returns > 0]

# On récupère les tickers des index avec rendements positifs
tickers_pos = positive_returns.index.to_list()

Actualisons le dataframe prices avec uniquement les tickers avec un rendement positif

In [56]:
prices = prices[tickers_pos]

## Analyse des Volatilités

Nous calculons la volatilites des actions avec la fonction asset_vol.

In [57]:
# Calculons la volatilité annuelle de chaque actif et trions les valeurs par ordre décroissant
total_vol = pf.asset_vol(prices).sort_values(ascending=True)
total_vol

ROOD.AS        0.748127
AGN.AS        12.640179
EAS2P.AS      14.934748
CTAC.AS       16.252868
IEX.AS        17.617157
VALUE.AS      18.694897
PORF.AS       21.044427
VTA.AS        23.193275
SBMO.AS       32.508720
TOM2.AS       33.287206
INGA.AS       37.973304
HEIJM.AS      41.899145
BRNL.AS       44.420426
VASTN.AS      52.496398
SNOW.AS       54.752081
BGHL.AS       62.296180
AJAX.AS       62.672310
VLK.AS        67.076211
SHELL.AS      67.304495
ACOMO.AS      91.739262
SLIGR.AS      95.395512
BRILL.AS      97.163977
VPK.AS        99.915111
KENDR.AS     104.703251
AD.AS        107.982181
REN.AS       114.713417
ARCAD.AS     138.583458
CRBN.AS      156.809416
AMG.AS       181.959332
RAND.AS      186.146327
AALB.AS      189.898765
TWEKA.AS     208.263251
NEDAP.AS     240.468437
HEIO.AS      330.274225
BESI.AS      333.216240
AKZA.AS      343.236776
HYDRA.AS     358.546970
HEIA.AS      364.857634
WKL.AS       445.673250
HAL.AS       524.112975
DSM.AS       710.757258
HOLCO.AS     730

In [59]:
tickers_vol = total_vol.index.to_list()[:35]
print(f'Nombre de tickers sélectionnés : {len(tickers_vol)}')

Nombre de tickers sélectionnés : 35


## Critères ESG

Nous allons créer notre portefeuille d'actions en se basant sur les critères ESG. Comme vu en cours cela s'appelle un ESG filter :
* exclude x% of firms with the worts ESG score
* keep firms with the best ESG momentum
* take a specific KPI

In [60]:
# récupérons les données liées aux critères ESG sauvegardés dans le fichier esg_scores.csv
esg = pd.read_csv('datas/esg_scores.csv', sep=',')

# supprimons les lignes tickers avec un return négatif
esg = esg[esg['Ticker Yahoo'].isin(tickers_vol)]

# On ne conserve que les tickers avec des scores ESG différents de NaN
esg = esg.dropna()

n = len(esg)

print(f'Nombre de tickers avec des scores ESG : {n}')

Nombre de tickers avec des scores ESG : 20


In [61]:
# On conserve les tickers avec un score environnemental supérieur à la médiane
esg = esg.sort_values(by=['Environment Score'])
tickers_env = esg.iloc[int(n*1/4):]
tickers_env

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
57,57,TWEKA.AS,49.0,69.0,67.0,61.0
60,60,VASTN.AS,52.0,45.0,74.0,57.0
15,15,BESI.AS,61.0,69.0,78.0,69.0
48,48,RAND.AS,63.0,81.0,76.0,75.0
7,7,AMG.AS,65.0,71.0,92.0,74.0
2,2,AD.AS,68.0,81.0,88.0,80.0
0,0,AALB.AS,71.0,75.0,58.0,69.0
62,62,VPK.AS,76.0,79.0,51.0,70.0
30,30,HEIO.AS,76.0,78.0,56.0,72.0
19,19,CRBN.AS,78.0,75.0,68.0,74.0


In [62]:
# On conserve les tickers avec un score social supérieur à la médiane
esg = esg.sort_values(by=['Social Score'])
tickers_soc = esg.iloc[int(n*1/4):]
tickers_soc

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
57,57,TWEKA.AS,49.0,69.0,67.0,61.0
15,15,BESI.AS,61.0,69.0,78.0,69.0
35,35,KENDR.AS,39.0,70.0,60.0,57.0
7,7,AMG.AS,65.0,71.0,92.0,74.0
0,0,AALB.AS,71.0,75.0,58.0,69.0
19,19,CRBN.AS,78.0,75.0,68.0,74.0
30,30,HEIO.AS,76.0,78.0,56.0,72.0
62,62,VPK.AS,76.0,79.0,51.0,70.0
2,2,AD.AS,68.0,81.0,88.0,80.0
48,48,RAND.AS,63.0,81.0,76.0,75.0


In [63]:
# Concaténation des dataframes en supprimant les doublons
tickers_esg = pd.merge(tickers_env, tickers_soc, how ='inner', on =['Ticker Yahoo'])
tickers_esg

,A_x,Ticker Yahoo,Environment Score_x,Social Score_x,Governance Score_x,Total Score_x,A_y,Environment Score_y,Social Score_y,Governance Score_y,Total Score_y
0,57,TWEKA.AS,49.0,69.0,67.0,61.0,57,49.0,69.0,67.0,61.0
1,15,BESI.AS,61.0,69.0,78.0,69.0,15,61.0,69.0,78.0,69.0
2,48,RAND.AS,63.0,81.0,76.0,75.0,48,63.0,81.0,76.0,75.0
3,7,AMG.AS,65.0,71.0,92.0,74.0,7,65.0,71.0,92.0,74.0
4,2,AD.AS,68.0,81.0,88.0,80.0,2,68.0,81.0,88.0,80.0
5,0,AALB.AS,71.0,75.0,58.0,69.0,0,71.0,75.0,58.0,69.0
6,62,VPK.AS,76.0,79.0,51.0,70.0,62,76.0,79.0,51.0,70.0
7,30,HEIO.AS,76.0,78.0,56.0,72.0,30,76.0,78.0,56.0,72.0
8,19,CRBN.AS,78.0,75.0,68.0,74.0,19,78.0,75.0,68.0,74.0
9,9,ARCAD.AS,78.0,90.0,73.0,81.0,9,78.0,90.0,73.0,81.0


In [64]:
# Récupérons la liste des tickers
list_esg = tickers_esg['Ticker Yahoo'].to_list()
print(f'Nombre de tickers ESG sélectionnés : {len(list_esg)}')

Nombre de tickers ESG sélectionnés : 13


## Enregistrement des tickers

Nous devons enregistrer les tickers sélectionnés dans le fichier tickers.txt

In [65]:
pf.tickers_storage(list_esg, 'datas/tickers.txt')